## 模型训练与评估

本章节将详细介绍本项目中各模型的训练方法和评估策略。

模型类型：
- **Transformer**: 基础的Transformer模型
- **EmoPrepend**: 情感前置模型，将情感信息添加到输入前
- **EmpDG_woD**: 无判别器的共情对话生成模型（仅生成器）
- **EmpDG_woG**: 无生成器的模型（使用EmoPrepend作为生成器，仅训练判别器）
- **EmpDG**: 完整的EmpDG模型（包含生成器和判别器的对抗训练）

训练脚本：
- `train.py`: 基础模型训练脚本
- `adver_train.py`: 完整EmpDG模型的对抗训练脚本
- `adver_train_no_eg.py`: 无多分辨率情感感知的对抗训练脚本

### 1. 模型训练

#### 1.1 训练参数说明

完整的参数请查看 `./utils/config.py`，这里仅展示最主要的参数设置。

**核心参数：**
| 参数 | 说明 | 默认值 | 备注 |
|------|------|--------|------|
| `--model` | 模型类型 | - | Transformer/EmoPrepend/EmpDG_woD/EmpDG/EmpDG_woG |
| `--cuda` | 使用GPU | False | 建议开启以加速训练 |
| `--device_id` | GPU设备ID | 0 | 指定使用的GPU |
| `--batch_size` | 批次大小 | 16 | 根据GPU内存调整 |
| `--epoch` | 最大训练轮数 | 100 | 有早停机制 |

**模型结构参数：**
| 参数 | 说明 | 默认值 | 备注 |
|------|------|--------|------|
| `--emb_dim` | 词嵌入维度 | 300 | 与GloVe向量维度保持一致 |
| `--hidden_dim` | 隐藏层维度 | 300 | Transformer模型的隐藏维度 |
| `--rnn_hidden_dim` | RNN隐藏维度 | 300 | 对于包含RNN的模型 |
| `--hop` | Transformer层数 | 1 | 编码器/解码器层数 |
| `--heads` | 多头注意力头数 | 2 | 注意力机制的头数 |

**训练策略参数：**
| 参数 | 说明 | 默认值 | 备注 |
|------|------|--------|------|
| `--noam` | Noam学习率调度 | False | 使用Transformer的学习率调度策略 |
| `--label_smoothing` | 标签平滑 | False | 缓解过拟合 |
| `--pretrain_emb` | 预训练词嵌入 | False | 使用GloVe词向量初始化 |
| `--pointer_gen` | 指针生成网络 | False | 允许模型从输入中复制词汇 |

**对抗训练参数：**
| 参数 | 说明 | 默认值 | 备注 |
|------|------|--------|------|
| `--d_steps` | 判别器训练步数 | 1 | 每轮中判别器的更新次数 |
| `--g_steps` | 生成器训练步数 | 5 | 每轮中生成器的更新次数 |
| `--emotion_disc` | 情感判别器 | False | 是否使用情感判别器 |
| `--resume_g` | 恢复生成器 | False | 从检查点恢复生成器训练 |
| `--resume_d` | 恢复判别器 | False | 从检查点恢复判别器训练 |

**其他参数：**
| 参数 | 说明 | 默认值 | 备注 |
|------|------|--------|------|
| `--save_path` | 保存路径 | - | 模型权重和日志保存位置 |
| `--test` | 测试模式 | False | 跳过训练，直接测试 |
| `--specify_model` | 指定模型 | False | 使用特定模型路径进行测试 |
| `--resume_path` | 恢复路径 | - | 指定模型的检查点路径 |

#### 1.2 模型训练示例

**模型训练命令示例：**
1. Transformer基础模型
    ```bash
    python train.py --cuda --label_smoothing --noam --emb_dim 300 --hidden_dim 300 \
        --hop 1 --heads 2 --pretrain_emb --model Transformer --device_id 0 \
        --save_path results/tb_results/Transformer/ --pointer_gen
    ```
2. EmoPrepend模型
    ```bash
    python train.py --cuda --label_smoothing --noam --emb_dim 300 --hidden_dim 300 \
        --hop 1 --heads 2 --pretrain_emb --model EmoPrepend --device_id 0 \
        --save_path results/tb_results/EmoPrepend/ --pointer_gen
    ```
3. EmpDG_woD模型（无判别器的共情生成器）
    ```bash
    python train.py --cuda --label_smoothing --noam --emb_dim 300 --hidden_dim 300 \
        --hop 1 --heads 2 --pretrain_emb --model EmpDG_woD --device_id 0 \
        --save_path results/tb_results/EmpDG_woD/ --pointer_gen
    ```
4. EmpDG_woG模型（判别器，使用EmoPrepend作为生成器）
    ```bash
    python3 adver_train_no_eg.py --cuda --label_smoothing --noam --emb_dim 300 \
        --rnn_hidden_dim 300 --hidden_dim 300 --hop 1 --heads 2 --pretrain_emb \
        --model EmpDG_woG --device_id 0 --save_path results/tb_results/EmpDG_woG/ \
        --d_steps 1 --g_steps 5 --pointer_gen
    ```
5. EmpDG完整模型对抗训练
    ```bash
    python3 adver_train.py --cuda --label_smoothing --noam --emb_dim 300 \
        --rnn_hidden_dim 300 --hidden_dim 300 --hop 1 --heads 2 --emotion_disc \
        --pretrain_emb --model EmpDG --device_id 0 --save_path results/tb_results/EmpDG/ \
        --d_steps 1 --g_steps 5 --pointer_gen
    ```
6. 使用预训练模型继续训练
    ```bash
    python3 adver_train.py --cuda --label_smoothing --noam --emb_dim 300 \
        --rnn_hidden_dim 300 --hidden_dim 300 --hop 1 --heads 2 --emotion_disc \
        --pretrain_emb --model EmpDG --device_id 0 --save_path results/tb_results/EmpDG/ \
        --d_steps 1 --g_steps 5 --pointer_gen --resume_g --resume_d
    ```

对抗训练是本项目的核心特色，包含生成器和判别器的对抗学习过程。

使用 adver_train.py 进行完整的EmpDG模型训练，包含以下三个阶段：

阶段1：预训练共情生成器
- 训练一个能够生成共情响应的生成器
- 使用交叉熵损失和其他监督信号

阶段2：预训练判别器
- **语义判别器**: 区分生成响应和真实响应的语义质量
- **情感判别器**: 区分生成响应和真实响应的情感适配性

阶段3：联合对抗训练
- 生成器和判别器进行对抗训练
- 生成器试图"欺骗"判别器
- 判别器试图"识别"生成的响应

训练策略
- **判别器步数(d_steps)**: 每轮训练中判别器的更新次数 (默认1次)
- **生成器步数(g_steps)**: 每轮训练中生成器的更新次数 (默认5次)
- **早停机制**: 基于验证集准确率进行早停

下面我们设置`epoch=1`来进行训练测试，以跑通整个算法流程（完整训练请训练上述完整模型训练的指令）：

In [1]:
from run_train import train_notebook

result = train_notebook(
    epochs=1,
    emb_dim=300,
    hidden_dim=300,
    hop=1,
    heads=2,
    pretrain_emb=True,
    label_smoothing=True,
    noam=True,
    pointer_gen=True,
    save_path="results/tb_results/EmoPrepend/",
    enable_test=True,      # 执行测试
    verbose_test=False,    # 静默测试
    model="EmoPrepend",
)

print(result)

['e:\\Study\\Programs\\Anaconda3\\envs\\empdg\\python38.zip', 'e:\\Study\\Programs\\Anaconda3\\envs\\empdg\\DLLs', 'e:\\Study\\Programs\\Anaconda3\\envs\\empdg\\lib', 'e:\\Study\\Programs\\Anaconda3\\envs\\empdg', '', 'C:\\Users\\惠普\\AppData\\Roaming\\Python\\Python38\\site-packages', 'e:\\Study\\Programs\\Anaconda3\\envs\\empdg\\lib\\site-packages', 'e:\\Study\\Programs\\Anaconda3\\envs\\empdg\\lib\\site-packages\\win32', 'e:\\Study\\Programs\\Anaconda3\\envs\\empdg\\lib\\site-packages\\win32\\lib', 'e:\\Study\\Programs\\Anaconda3\\envs\\empdg\\lib\\site-packages\\Pythonwin', 'e:\\Study\\Programs\\Anaconda3\\envs\\empdg\\lib\\site-packages\\setuptools\\_vendor', '/apdcephfs/share_916081/qtli/install/ft_local/EmpDG']
开始训练 EmoPrepend 模型...
配置: emb_dim=300, hidden_dim=300, epochs=1
加载数据...
LOADING empathetic_dialogue ...
[situation]: i spent all weekend working on my truck to fix a miss in the engine . despite spending over $ 200 in parts , it did not do a thing to fix the miss .
[emotio

100%|██████████| 1296/1296 [05:58<00:00,  3.62it/s]


训练完成!
开始测试...
testing generation (quiet mode)...


loss:9.6987; ppl:16297.2: 100%|██████████| 2713/2713 [26:17<00:00,  1.72it/s]

测试完成! Test Loss: 9.6987, PPL: 24725.8045, Acc: 0.0291; Dist1: 0.0068; Dist2: 0.0213
{'training_completed': True, 'test_performed': True, 'loss': 9.698747108134668, 'ppl': 24725.804485522283, 'accuracy': 0.029119056395134537, 'best_val_ppl': 1000, 'dist1': 0.006772041658535881, 'dist2': 0.02128459197906955}


In [ ]:
from run_adver_train import adver_train_notebook

result = adver_train_notebook(
    epochs=1,
    emb_dim=300,
    hidden_dim=300,
    hop=1,
    heads=2,
    pretrain_emb=True,
    label_smoothing=True,
    noam=True,
    pointer_gen=True,
    save_path="results/tb_results/EmpDG/",
    resume_g=True,
    resume_d=True,
    enable_test=False,      # 执行测试
    verbose_test=False,    # 静默测试
    model="EmpDG",
)

print(result)

加载数据 (adversarial)...
LOADING empathetic_dialogue ...
=====================STEP 1: Pre-train Empathetic Generator=====================
Embeddings: 22359 x 300
Loading embedding file: vectors/glove.6B.300d.txt
Pre-trained: 18064 (80.79%)


e:\Study\ml_projects\EmpDG-master\adver_train.py:104: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('result/best_backup/EmpDG_woD_best.tar', map_loca

=====================STEP 2: Pre-train Discriminators==========================
Embeddings: 22359 x 300
Loading embedding file: vectors/glove.6B.300d.txt
Pre-trained: 18064 (80.79%)
Embeddings: 22359 x 300
Loading embedding file: vectors/glove.6B.300d.txt
Pre-trained: 18064 (80.79%)
=====================STEP 3: Adversarial joint learning=======================
==================Test performance before adver train=====================
LOADING empathetic_dialogue ...


e:\Study\ml_projects\EmpDG-master\adver_train.py:210: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('result/best_backup/D_best.tar', map_location=lam

Loss	PPL	Accuracy	Dist-1	Dist-2
3.6748	39.4426	0.3349	0.70	6.26


loss:3.7104; ppl:40.9: 100%|██████████| 186/186 [09:13<00:00,  2.98s/it]



adver_training !
EVAL	Loss	PPL	Accuracy	Dist-1	Dist-2
valid	3.7104	40.8687	0.3491	0.68	6.70
best_acc: 0.3491263440860215; patient: 0
step 200 spend time: 557.036807


loss:3.8415; ppl:46.6: 100%|██████████| 186/186 [09:02<00:00,  2.92s/it]



adver_training !
EVAL	Loss	PPL	Accuracy	Dist-1	Dist-2
valid	3.8415	46.5954	0.3150	0.70	6.06
{'training_completed': True, 'test_performed': False}


### 2. 模型评估

这里主要采用 **Perplexity、Distinct-1、Distinct-2、Emotion Accuracy** 这四个自动指标，对模型的性能进行综合性评估。

#### 2.1 Perplexity（困惑度，PPL）

**定义 / 公式**
给定测试集中所有目标 token 的总数 $N$，模型按条件概率 $p(y_j\mid y_{<j},C)$（通常用 teacher-forcing 的方式计算），困惑度定义为：

$$
\mathrm{PPL} = \exp\Big(-\frac{1}{N}\sum_{j=1}^{N}\log p(y_j\mid y_{<j},C)\Big).
$$

（也常写成 $\mathrm{PPL} = \exp(\text{平均负对数似然})$）

**实际计算要点**

* 用**真值目标序列**来计算概率（即在每一步使用 gold 前缀，teacher-forcing），而不是用模型自己生成的序列来估概率。
* 在实现上通常把 batch 内的对数概率累加后归一化到全数据上的负对数似然，再取指数。
* 若你在训练时应用了 label-smoothing 或其他正则，要注意 PPL 的可比性：和 baseline 要一致设置。

**EmpDG 中的含义 / 解读**

* PPL 测量语言模型层面的“流畅性 / 模型对真实回复的拟合程度”。
* 在 EmpDG 中，引入对抗判别器往往会 **提高多样性**（Distinct↑）但可能 **使模型的最大似然概率下降**，因此 PPL 反而变差（变大）。这并不一定意味着生成质量变差 —— 只是概率分布发生了变化（更分散、更多样）。论文中也观察到这种 trade-off。

**优点**

* 标准、可重复，衡量的是模型对训练/测试语料的概率拟合能力。

**局限**

* 对多模态回复（多个合理答案）不友好：真实的高质量但“少见”回复会提高 PPL。
* 不能直接衡量回复的情感/共情性或多样性。

#### 2.2 & 2.3 Distinct-1 / Distinct-2（表面多样性）

**定义）**
对整个测试集上所有**生成的回复**（不是 gold），统计不重复的 n-gram 数：

$$
\text{Distinct-}n = \frac{\#\ \text{distinct n-grams in all outputs}}{\#\ \text{total generated tokens (or total n-grams)} }.
$$

**实际计算要点**

* 在计算时只统计 *模型生成的* 文本，不包括 gold。
* 典型做法是把 test 集上所有生成回复拼在一起统计 distinct，得到 corpus-level distinct；也可做 sentence-level distinct 然后平均（两者不同，paper 通常用 corpus-level）。
* 区分 unigram（n=1）和 bigram（n=2），分别报告 Distinct-1/2。

**EmpDG 中的含义**

* Distinct-1/2 衡量的是**表面 n-gram 多样性**：值越大说明生成回复更不“千篇一律”。
* EmpDG 中对抗训练（interactive discriminators）鼓励生成器输出更“非典型”的回复，从而 Distinct-2 在论文里显著上涨，说明 bigram 级别的多样性增加（更丰富的短语组合）。
* 但高 Distinct 并不自动等同于“好”的回复——可能生成更多不同但不相关或不连贯的短语，所以必须结合 Relevance/Empathy 人工评分解读。

**优点**

* 直观、能显著反映“通用回复”问题的缓解效果（higher ≈ less generic）。

**局限**

* 只测表面多样性：不能衡量语义多样性或情感适配性。
* 对 rare token 的使用敏感：错误/拼写/长尾词也会提高 distinct。
* 规范化方式（是否乘 100、除以 tokens 还是 n-grams）会影响绝对值，比较时要一致。

#### 2.4 Emotion Accuracy（情感准确率）

**论文中具体做法（EmpDG 的做法）**
论文文本写法是：“Emotion Accuracy as the agreement between the ground truth emotion labels and the predicted emotion labels by the empathetic generator.”
在 EmpDG 的模型结构中，情感预测向量 $\mathbf{e}_p$ 是由 **情感 encoder（基于 context）** 预测得到的 coarse-grained 情感类别分布（公式 7、8）。因此 **Emotion Accuracy 在此处主要衡量模型从上下文识别并预测正确情感标签的能力（emotion recognition on context）**，即模型是否正确“理解”用户在上下文里表达的情绪类别。

> 小结：**不是直接比较生成回复的情绪表达本身**，而是比较模型对 **对话情感标签的预测** 与数据集给定标签的一致性。

**定义）**
令 $e^\ast$ 为数据集给定的对话级情感标签，令 $\hat{e} = \arg\max P_e(e\mid \mathcal{E})$ 为模型预测的情感类别，则：

$$
\text{EmotionAccuracy} = \frac{1}{M}\sum_{i=1}^M \mathbf{1}(\hat{e}^{(i)} = e^{\ast (i)})
$$

其中 $M$ 是测试样本数。

**为什么用这个指标（在 EmpDG 的设计下）**

* EmpDG 强调**multi-resolution emotion perception**（多分辨率情感感知），模型内部明确有一个情感分类器结构。Emotion Accuracy 测量这一模块是否有效，即模型是否能从对话上下文捕捉到正确的情绪信号，这是共情能力的**前置要素**（理解先于回应）。
* 如果模型不能正确识别情感，那么即使 decoder 输出表面上有情感词，也可能不是针对正确情绪（例如把“悲伤”误识为“惊喜”）。

**优点**

* 直接评估情感理解模块的准确性，是一个可重复、客观的度量。
* 在论文里能反映 multi-resolution encoder 的贡献（如果 EmotionAccuracy ↑，说明情感感知模块有效）。

**局限**

* **并不等于“生成的回复是否表现出共情”**：一个模型可以正确预测情感标签（EmotionAccuracy 高），但生成的回复依然可能不合适或不共情（这就是为什么还需要人工的 Empathy 评分）。
* **依赖 gold label 的质量与标签颗粒度**：EmpatheticDialogues 的情感标签是 coarse-grained（有限类别），可能掩盖细微情绪差别。
* **评估范围有限**：只衡量“理解”而非“表达”。若要评价回复本身的情感表达，需另行用情感分类器对生成回复进行分类，或用人工评估。

**总结**

* **PPL** 衡量语言概率拟合（流畅性），但对多解场景不敏感；
* **Distinct-1/2** 直观反映表面多样性，能证明对抗训练减轻通用回复，但需结合相关性判断是否“有意义”的多样性；
* **Emotion Accuracy（论文中）** 衡量模型从上下文识别情绪的能力（context→emotion），是共情能力的必要前提，但**不等同于**生成回复是否真正表达或实现了共情。

In [1]:
from run_train import train_notebook

# 示例：仅运行测试（从 result/ 加载模型权重并在测试集上评估）
# 注意：请确保存在 result/<MODEL>_best.tar（如 result/EmoPrepend_best.tar）
result = train_notebook(
    emb_dim=300,
    hidden_dim=300,
    hop=1,
    heads=2,
    pretrain_emb=True,
    label_smoothing=True,
    noam=True,
    pointer_gen=True,
    test=True,            # 进入测试模式：从 result/ 读取模型权重
    enable_test=True,     # 测试完成后返回指标
    verbose_test=False,   # 若为 True 会打印每个样例的详细生成
    model='EmoPrepend',
)

print(result)

['e:\\Study\\Programs\\Anaconda3\\envs\\empdg\\python38.zip', 'e:\\Study\\Programs\\Anaconda3\\envs\\empdg\\DLLs', 'e:\\Study\\Programs\\Anaconda3\\envs\\empdg\\lib', 'e:\\Study\\Programs\\Anaconda3\\envs\\empdg', '', 'C:\\Users\\惠普\\AppData\\Roaming\\Python\\Python38\\site-packages', 'e:\\Study\\Programs\\Anaconda3\\envs\\empdg\\lib\\site-packages', 'e:\\Study\\Programs\\Anaconda3\\envs\\empdg\\lib\\site-packages\\win32', 'e:\\Study\\Programs\\Anaconda3\\envs\\empdg\\lib\\site-packages\\win32\\lib', 'e:\\Study\\Programs\\Anaconda3\\envs\\empdg\\lib\\site-packages\\Pythonwin', 'e:\\Study\\Programs\\Anaconda3\\envs\\empdg\\lib\\site-packages\\setuptools\\_vendor', '/apdcephfs/share_916081/qtli/install/ft_local/EmpDG']
开始训练 EmoPrepend 模型...
配置: emb_dim=300, hidden_dim=300, epochs=100
加载数据...
LOADING empathetic_dialogue ...
[situation]: i spent all weekend working on my truck to fix a miss in the engine . despite spending over $ 200 in parts , it did not do a thing to fix the miss .
[emot

e:\Study\ml_projects\EmpDG-master\run_train.py:305: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('result/' + config.model + '_best.tar', map_locatio

MODEL USED EmoPrepend
TRAINABLE PARAMETERS 14496563
测试模式...
testing generation (quiet mode)...


loss:9.6987; ppl:16297.2: 100%|██████████| 2713/2713 [24:40<00:00,  1.83it/s]

Test Loss: 9.6987, PPL: 24725.8045, Acc: 0.0291; Dist1: 0.0068; Dist2: 0.0213
None
